# Úvodné nastavenie

## Načítanie knižníc

In [4]:
from pathlib import Path
import pandas as pd # type: ignore
import numpy as np # type: ignore
from datetime import datetime


In [5]:
# Aby fungovalo nacitanie spolocnych funkcii, je nutne ich nainstalovat cez `pip install .` na urovni, kde sa nachadza pyproject.toml

from OSN_common.nacitanie_dat import nacitaj_data_zp, nacitaj_prevodove_subory
from OSN_common.pomocne_funkcie import zjednot_kod

## Konfigurácia výstupných súborov

In [6]:
_paths = {
    "vystupy": Path('.') / "Vystupy"
}

_datum = datetime.now().strftime("%Y%m%d_%H%M%S")

_paths['vystupy'] = _paths['vystupy'] / _datum
_paths['vystupy'].mkdir()

_paths['validacia'] = _paths['vystupy'] / 'validacia'
_paths['validacia'].mkdir()

print(f'Ukladam data do priecinku {_paths["vystupy"]}')

Ukladam data do priecinku Vystupy/20240717_105809


# Načítanie dát

In [7]:
_rok = 2023
_spracovat = [24, 25, 27]

_data = nacitaj_data_zp(_rok, _spracovat)


Nacitavam data pre poistovnu dovera
Nacitavam data zo suboru 2023_24_01_UZS_JZS_oprava.csv
Nacitavam data zo suboru 2023_24_02_HP_oprava.csv
Nacitavam data zo suboru 2023_24_03_PREKLAD_oprava.csv
Nacitavam data zo suboru 2023_24_04_VDG.csv
Nacitavam data zo suboru 2023_24_05_VYKON.csv
Nacitavam data zo suboru 2023_24_09_POISTENCI_oprava.csv
Nacitavam data zo suboru 2023_24_13_SUMAR.csv
Nacitavam data pre poistovnu vzp
Nacitavam data zo suboru 2023_25_01_UZS_JZS_oprava.csv
Nacitavam data zo suboru 2023_25_02_HP_oprava.csv
Nacitavam data zo suboru 2023_25_03_PREKLAD.csv
Nacitavam data zo suboru 2023_25_04_VDG.csv
Nacitavam data zo suboru 2023_25_05_VYKON.csv
Nacitavam data zo suboru 2023_25_09_POISTENCI_oprava.csv
Nacitavam data zo suboru 2023_25_09_POISTENCI_doplnenie.csv
Nacitavam data zo suboru 2023_25_13_SUMAR.csv
Nacitavam data pre poistovnu union
Nacitavam data zo suboru 2023_27_01_UZS_JZS_oprava.csv
Nacitavam data zo suboru 2023_27_02_HP_oprava.csv
Nacitavam data zo suboru 2023_27

# Predspracovanie dát

## Priprav dat UZS_JZS (`uzs`)

### Predspracuj data zo suboru 01_UZS_JZS

In [8]:
# Najdi relevantne tabuľky
_uzs = pd.concat([_data[k]['uzs_jzs'] for k in _spracovat], ignore_index=True)

len(_uzs)

1317575

In [9]:
# Vyluc riadky, kde typ hospitalizacie je "pripocitatelna polozka"
_uzs = _uzs[_uzs['typ_hospitalizacie'] != 'Z']

_uzs = _uzs.drop(columns=['typ_hospitalizacie'])

len(_uzs)

1317574

In [10]:
# Exportuj riadky, kde chyba ID hospitalizacie a typ je DRG alebo UH.
_reportovane_stlpce = ['kod_zp','id_hp']
_argumenty_reportu = {'sep': ';', 'index': False}

_uzs[_uzs['id_hp'].isna() & _uzs['typ_starostlivosti'].isin(['DRG', 'UH'])][_reportovane_stlpce + ['id_poistenca', 'typ_starostlivosti', 'pzs_12', 'obdobie']].to_csv(_paths['validacia'] / 'id_hp_chybajuce_01.csv', **_argumenty_reportu)


In [11]:
# Prirad cislo riadku ako ID_HP pre chybajuce ID
# Takato hospitalizacia sa neda naparovat na ostatne subory (nie je problem v pripade JZS), ale je to lepsie ako ich vyhodit
_uzs.loc[_uzs.id_hp.isna(), 'id_hp'] = 'x' + _uzs[_uzs.id_hp.isna()].index.map(str)

In [12]:
# Ocisti kody vykonov a diagnoz na standardizovany tvar - len male pismena a cisla.

_stlpce_s_kodmi = ['kod_operacneho_vykonu', 'kod_jednodnoveho_vykonu', 'dgn_prijem', 'dgn_prepustenie']

_uzs[_stlpce_s_kodmi] = _uzs[_stlpce_s_kodmi].apply(lambda col: col.apply(zjednot_kod))

In [13]:
# Chceme pocitat pocet spravenych vykonov, tak vytvor kombinaciu vykon-obdobie,
# aby sa vykon spravne zapocital viackrat, ak bol pocas jednej hospitalizacie
# robeny viackrat.
# Vytvor zoznam vykonov v tvare, ako ho pouzivame pre algoritmus. 
# Kedze uzs data neobsahuju datum vykonu, iba obdobie, v ktorom bola dodana starostlivost, pouzi namiesto dna xx
# Kedze uzs data neobsahuju lokalizaciu vykonu, pouzi kod Z "neznama lokalizacia"

_uzs['operacny_vykon'] = _uzs['kod_operacneho_vykonu'] + '&Z&' + _uzs['obdobie'].str.replace('-', '') + 'xx'
_uzs['jednodnovy_vykon'] = _uzs['kod_jednodnoveho_vykonu'] + '&Z&' + _uzs['obdobie'].str.replace('-', '') + 'xx'

_uzs = _uzs.drop(columns=['kod_operacneho_vykonu', 'kod_jednodnoveho_vykonu'])


### Zjednoť dáta v `uzs`

In [14]:
# Zjednot celu tabulku tak, aby id_hp bolo unikatnym identifikatorom (kazdy riadok samostatne id_hp).

# Pomocná funkcia na jednoriadkový zápis odstránenia prvku z množiny
def _discard(s, e):
    s.discard(e)
    return s

# pomocná funkcia na jednoriadkový zápis odstránenia nan hodnôt
def _dropna(a):
    return a[pd.notnull(a)]

# pomocna funkcia na vlozenie prvku na zaciatok
def _insert_first(l, a):
    l.insert(0, a)
    return l

# Tabuľka `uzs` zoskupená podľa id_hp, na ktorom sa budú agregovať jednotlivé stĺpce
_grouped_uzs = _uzs.sort_values(by=['obdobie', 'datum_od', 'datum_do'], ascending=True, na_position='first').groupby(by=['kod_zp', 'id_hp'])

# Tabuľka, do ktorej sa budú ukladať výstupné zjednotenia
zjednotene_uzs = pd.DataFrame()

In [15]:
# 'datum_od' vezmi najmensi
# 'datum_do' vezmi najvacsi
# 'obdobie_od' vezmi najmensie obdobie
# 'obdobie_do' vezmi najvacsie obdobie
# (Mozeme si dovolit pouzit first a last kedze je to zoradene, min a max su o dost pomalsie)

zjednotene_uzs['datum_od'] = _grouped_uzs['datum_od'].first()
zjednotene_uzs['datum_do'] = _grouped_uzs['datum_do'].last()
zjednotene_uzs['obdobie_od'] = _grouped_uzs['obdobie'].first()
zjednotene_uzs['obdobie_do'] = _grouped_uzs['obdobie'].last()

In [16]:
# vykony: vytvor zoznam vykonanych vykonov

zjednotene_uzs['vykony'] =  _grouped_uzs[['operacny_vykon', 'jednodnovy_vykon']].apply(lambda z: '~'.join(_dropna(z.values.flatten()))).replace('', pd.NA)


In [17]:
# diagnozy: 
#   posledna znama diagnoza pri prepusteni je hlavna diagnoza
#       ak nebola ziadna dianoza pri prepusteni, vezmi poslednu znamu diagnozu pri prijati 
#   vytvor mnozinu vsetkych diagnoz, bude obsahovat aj hlavnu diagnozu na zaciatku

zjednotene_uzs['posledna_dgn_prepustenie'] = _grouped_uzs['dgn_prepustenie'].last()
zjednotene_uzs['posledna_dgn_prijem'] = _grouped_uzs['dgn_prijem'].last()
zjednotene_uzs['hlavna_diagnoza'] = zjednotene_uzs['posledna_dgn_prepustenie'].fillna(zjednotene_uzs['posledna_dgn_prijem'])
zjednotene_uzs = zjednotene_uzs.drop(columns=['posledna_dgn_prepustenie', 'posledna_dgn_prijem'])

zjednotene_uzs['diagnozy'] = _grouped_uzs[['dgn_prijem', 'dgn_prepustenie']].apply(lambda z: set(_dropna(z.values.flatten())))
zjednotene_uzs['diagnozy'] = zjednotene_uzs[['hlavna_diagnoza', 'diagnozy']].apply(lambda r: '~'.join(_insert_first(list(_discard(r['diagnozy'], r['hlavna_diagnoza'])), r['hlavna_diagnoza'])), axis=1)

In [18]:
# 'typ_starostlivosti', 'pzs_12 a 'pzs_8' vezmi prve
# 'id_poistenca' vezmi posledne (stavalo sa pri porodoch, ze jedno id_hp, malo dvoch poistencov, kedze najprv to bolo pisane na matku az neskor na dieta)

zjednotene_uzs['typ_starostlivosti'] = _grouped_uzs['typ_starostlivosti'].first()
zjednotene_uzs['pzs_12'] = _grouped_uzs['pzs_12'].first()
zjednotene_uzs['id_poistenca'] = _grouped_uzs['id_poistenca'].last()

In [19]:
# Vypis statistiky

# napoveda k nazvom stlpcov v povodnom R kode:
# ID_POI_ZP = id_poistenca
# ID_HOSP_ZP = id_hp
# DAT_PRIJ = datum_od
# DAT_PREP = datum_do
# KOD_VYK_OPER = kod_operacneho_vykonu
# TYP_ZS = typ_hospitalizacie
# ins_name = kod_zp
# nemP01 = pzs_8
# id_hosp_nem = id_hp_pzs_8
# diag_uzs = dgn_prijem
# vykon_uzs = kod_vykonu_obdobie

print(len(zjednotene_uzs))
print(zjednotene_uzs.reset_index().nunique())

1194663
kod_zp                      3
id_hp                 1194663
datum_od                  597
datum_do                  365
obdobie_od                 18
obdobie_do                 13
vykony                  32880
hlavna_diagnoza          8569
diagnozy                96512
typ_starostlivosti          4
pzs_12                   1995
id_poistenca           716567
dtype: int64


## Priprava dat DRG

### Priprav data zo suboru 02_HP_UDAJE

In [20]:
# Najdi relevantne tabulky
_drg_full = pd.concat([_data[i]['hp'] for i in _spracovat], ignore_index=True)

len(_drg_full)

798355

In [21]:
# Vyhod nepotrebne stlpce (v tomto pripade ziadne) a duplikaty

# napoveda k nazvom stlpcov v povodnom R subore
# ID_HP_DZP = id_hp
# vek_zapis = vek_roky
# HMOTNOST = hmotnost
# UPV = upv
# KOD_DIAG_HL = hlavna_diagnoza
# KOD_DRG = drg
# DAT_OD = datum_od
# DAT_DO = datum_do
# KOD_PZS = pzs_6
# TYP_ZS = typ_starostlivosti
# ins_name = kod_zp
# ID_HP_PZS = id_hp_pzs

drg = _drg_full.drop_duplicates()

print(len(drg))
print(drg.nunique())


798355
kod_zp                     3
id_hp_pzs             113098
id_hp                 798355
id_poistenca          514731
pzs_6                     90
datum_od              258433
datum_do              160193
osetrovacia_doba         172
vek_dni                  367
vek_roky                 104
hmotnost                2177
upv                      992
datum_narodenia        35593
hlavna_diagnoza         7569
drg                     1144
erv                    14721
zlucene_hp              8589
typ_starostlivosti         2
dtype: int64


### Priprav data zo suboru 03_HP_PREKLADY

In [22]:
# vyber relevantne tabulky
preklady = pd.concat([_data[i]['preklady'] for i in _spracovat], ignore_index=True)

print(len(preklady))
print(preklady.nunique())

984556
kod_zp           3
id_hp       798019
pzs_12        1250
datum_od       503
datum_do       468
dtype: int64


In [23]:
# Priprav na pripojenie k suboru 02_HP_UDAJE
# vytvor pzs_6 ako prvych 6 cislic z pzs_12
# Prirad prve pzs_12 na ktore bolo hp prijate a koresponduje s uvedenym pzs_6 

preklady['pzs_6'] = preklady['pzs_12'].str[:6]

_grouped_preklady = preklady.sort_values(by=['datum_od', 'datum_do'], ascending=True).groupby(by=['kod_zp', 'id_hp', 'pzs_6'])
drg = drg.merge(_grouped_preklady['pzs_12'].first(), on=['kod_zp', 'id_hp', 'pzs_6'])

print(len(drg))
print(drg.nunique())

798019
kod_zp                     3
id_hp_pzs             113097
id_hp                 798019
id_poistenca          514601
pzs_6                     90
datum_od              258433
datum_do              160193
osetrovacia_doba         172
vek_dni                  367
vek_roky                 104
hmotnost                2177
upv                      992
datum_narodenia        35593
hlavna_diagnoza         7569
drg                     1144
erv                    14719
zlucene_hp              8589
typ_starostlivosti         2
pzs_12                  1238
dtype: int64


In [24]:
# Exportuj preklady, ktore pre rovnaky id_hp maju zaznamenany preklad medzi roznymi poskytovatelmi

_pocty_pzs6 = preklady.groupby(['kod_zp', 'id_hp'])['pzs_6'].nunique()

preklady.set_index(['kod_zp', 'id_hp']).loc[_pocty_pzs6[_pocty_pzs6 > 1].index].sort_values('datum_do').to_csv(_paths['validacia'] / 'preklady_rozni_poskytovatelia.csv', **{**_argumenty_reportu, 'index': True})

In [25]:
# Exportuj zoznam hospitalizacii, ktore nie su zaznamenane v subore 03_PREKLADY

drg[drg['pzs_12'].isna()][_reportovane_stlpce + ['pzs_6']].to_csv(_paths['validacia'] / 'preklad_chybajuci.csv', **_argumenty_reportu)

### Priprav data zo suboru 04_HP_VDG

In [26]:
# Najdi relevantne tabulky

vdg = pd.concat([_data[i]['vdg'] for i in _spracovat], ignore_index=True)

len(vdg)

2087847

In [27]:
# Vyhod duplicity

vdg = vdg.drop_duplicates()

print(len(vdg))
print(vdg.nunique())

2018465
kod_zp         3
id_hp     574514
vdg        10897
dtype: int64


In [28]:
# Vytvor mnozinu vedlajsich diagnoz

vdg['vdg'] = vdg['vdg'].str.replace('[^0-9a-zA-Z]', '', regex=True).str.lower()
_vedlajsie_diagnozy = vdg.groupby(['kod_zp', 'id_hp'])['vdg'].apply(lambda z: '~'.join(z)).rename('vedlajsie_diagnozy')

In [29]:
# Pripoj vedlajsie diagnozy zo suboru 04_HP_VDG k suboru 02_HP, tam, kde neboli priradene pridaj prazdnu mnozinu (fillna(set()) nefunguje)

drg = drg.merge(_vedlajsie_diagnozy, how='left', on=['kod_zp', 'id_hp'])

print(len(drg))

798019


### Priprav data zo suboru 05_HP_ZV

In [30]:
# Vyber relevantne tabulky 

vykony = pd.concat([_data[i]['vykony'] for i in _spracovat], ignore_index=True)

In [31]:
# Vytvor identifikator vykonu na zaklade jeho kodu, datumu vykonania a
# lokalizacie. Tiez na zaklade jeho kodu, roku a mesiaca vykonania na porovnanie
# so suborom 01_UZS_JZS.

vykony.loc[:, ['kod_vykonu']] = vykony['kod_vykonu'].apply(zjednot_kod)
vykony.loc[:, ['kod_vykonu']] = vykony['kod_vykonu'] + '&' + vykony['lokalizacia_vykonu'].fillna('Z') + '&' + vykony['datum_vykonu'].str.replace('-','').fillna('xxxxxxxx')

In [32]:
# Vytvor zoznam vykonov a Pripoj vykony zo suboru 05_HP_ZV k suboru 02_HP

_zoznamy_vykonov = vykony.groupby(['kod_zp', 'id_hp'])['kod_vykonu'].apply(lambda z: '~'.join(z)).rename('vykony')

drg = drg.merge(_zoznamy_vykonov, how='left', on=['kod_zp', 'id_hp'])

print(len(drg))
print(drg.nunique())

798019
kod_zp                     3
id_hp_pzs             113097
id_hp                 798019
id_poistenca          514601
pzs_6                     90
datum_od              258433
datum_do              160193
osetrovacia_doba         172
vek_dni                  367
vek_roky                 104
hmotnost                2177
upv                      992
datum_narodenia        35593
hlavna_diagnoza         7569
drg                     1144
erv                    14719
zlucene_hp              8589
typ_starostlivosti         2
pzs_12                  1238
vedlajsie_diagnozy    317965
vykony                540511
dtype: int64


## Doplnenie DRG dat z UZS_JZS

In [33]:
# Pripoj data z 01_UZS_JZS k existujucim datam z 02_HP_UDAJE

final = drg.merge(zjednotene_uzs, how='left', on=['kod_zp', 'id_hp'], suffixes=['_02', '_01'])

print(len(final))

print(final.nunique())

798019
kod_zp                        3
id_hp_pzs                113097
id_hp                    798019
id_poistenca_02          514601
pzs_6                        90
datum_od_02              258433
datum_do_02              160193
osetrovacia_doba            172
vek_dni                     367
vek_roky                    104
hmotnost                   2177
upv                         992
datum_narodenia           35593
hlavna_diagnoza_02         7569
drg                        1144
erv                       14719
zlucene_hp                 8589
typ_starostlivosti_02         2
pzs_12_02                  1238
vedlajsie_diagnozy       317965
vykony_02                540511
datum_od_01                 455
datum_do_01                 365
obdobie_od                   17
obdobie_do                   13
vykony_01                 26004
hlavna_diagnoza_01         8120
diagnozy                  91718
typ_starostlivosti_01         2
pzs_12_01                  1242
id_poistenca_01          514602
d

###  Zjednot udaje zo suborov 01_UZS_JZS a 02_HP_UDAJE

Prioritu dostava subor 02_HP_UDAJE (`02`)

#### ID poistenca

In [34]:
final[final['id_poistenca_02'] != final['id_poistenca_01']][_reportovane_stlpce + ['id_poistenca_01','id_poistenca_02']].to_csv(_paths['validacia'] / 'id_poistenca_nezhodne.csv', **_argumenty_reportu)

final['id_poistenca'] = final['id_poistenca_02'].fillna(final['id_poistenca_01'])

final = final.drop(columns=['id_poistenca_01', 'id_poistenca_02'])

####   Typ zdravotnej starostlivosti

In [35]:
#   Dovera vykazuje veci svojsky, tak u nej vznikaju nezhody medzi typom ZS zo
#   suboru 01 a zo suboru 02. Pri zjednocovani do jedneho typu nam nezhody medzi
#   typom UH a DRG momentalne nevadia, ale ak je typ JZS podla suboru 02, tak to
#   prepise typ podla suboru 01.
#   Ak indikator chyba, prirad non-DRG typ UH.
#   Exportuj chybajuci typ starostlivosti a nezhodny typ starostlivosti


final[final['typ_starostlivosti_02'] != final['typ_starostlivosti_01']][_reportovane_stlpce + ['typ_starostlivosti_01', 'typ_starostlivosti_02']].to_csv(_paths['validacia'] / 'typ_starostlivosti_nezhodny.csv', **_argumenty_reportu)

final[final['typ_starostlivosti_02'] == 'UH'][_reportovane_stlpce + ['typ_starostlivosti_02']].to_csv(_paths['validacia'] / 'typ_starostlivosti_UH_v_02.csv', **_argumenty_reportu)

final['typ_starostlivosti'] = final['typ_starostlivosti_02'].fillna(final['typ_starostlivosti_01']).fillna('UH')

final = final.drop(columns=['typ_starostlivosti_01', 'typ_starostlivosti_02'])

#### Kod poskytovatela pzs_12

In [36]:
final[final['pzs_12_02'] != final['pzs_12_01']][_reportovane_stlpce + ['pzs_12_01', 'pzs_12_02']].to_csv(_paths['validacia'] / 'pzs_12_rozdielne.csv', **_argumenty_reportu)

final['pzs_12'] = final['pzs_12_02'].fillna(final['pzs_12_01'])

final = final.drop(columns=['pzs_12_01', 'pzs_12_02'])


#### Datumy

In [37]:
_nezhodne_datumy_od = final[['datum_od_01', 'datum_od_02']].notna().all(axis=1) & ((final['datum_od_02'] - final['datum_od_01']).dt.days != 0)
_nezhodne_datumy_do = final[['datum_do_01', 'datum_do_02']].notna().all(axis=1) & ((final['datum_do_02'] - final['datum_do_01']).dt.days != 0)
final[_nezhodne_datumy_do | _nezhodne_datumy_od][_reportovane_stlpce + ['datum_od_01','datum_od_02','datum_do_01','datum_do_02','typ_starostlivosti', 'obdobie_od', 'obdobie_do']].to_csv(_paths['validacia'] / 'datumy_nezhodne.csv', **_argumenty_reportu)

final['datum_od'] = final['datum_od_02'].fillna(final['datum_od_01'])
final['datum_do'] = final['datum_do_02'].fillna(final['datum_do_01'])

final = final.drop(columns=['datum_od_01', 'datum_do_01', 'datum_od_02', 'datum_do_02'])

#### Diagnozy

In [38]:
# kod hlavnej diagnozy a drg zjednot na standardny tvar male pismena, cisla 
final[['hlavna_diagnoza_02', 'drg']] = final[['hlavna_diagnoza_02', 'drg']].apply(lambda col: col.apply(zjednot_kod))

In [39]:
# Vyexportuj rozdielne a chybajuce hlavne diagnozy

final[final['hlavna_diagnoza_01'] != final['hlavna_diagnoza_02']][_reportovane_stlpce + ['hlavna_diagnoza_01', 'hlavna_diagnoza_02']].to_csv(_paths['validacia'] / 'hlavne_diagnozy_nezhodne.csv', **_argumenty_reportu)
final[final['hlavna_diagnoza_02'].isna()][_reportovane_stlpce + ['hlavna_diagnoza_02', 'hlavna_diagnoza_01']].to_csv(_paths['validacia'] / 'hlavne_diagnozy_chybajuce_02.csv', **_argumenty_reportu)

final['hlavna_diagnoza'] = final['hlavna_diagnoza_02'].fillna(final['hlavna_diagnoza_01'])

final = final.drop(columns=['hlavna_diagnoza_01', 'hlavna_diagnoza_02'])

In [40]:
# Ako vedlajsie diagnozy pouzi vedlajsie diagnozy z 02 a ponechaj pre analyticke ucely diagnozy z 01

final = final.rename(columns={'diagnozy': 'diagnozy_z_01'})

In [41]:
# Vytvor stlpec `diagnozy` obsahujuci hlavnu diagnozu na zaciatku

_spojka = np.where(final['vedlajsie_diagnozy'].notna(), '~', '')

final['diagnozy'] = final['hlavna_diagnoza'].fillna('') + _spojka + final['vedlajsie_diagnozy'].fillna('')

final = final.drop(columns=['hlavna_diagnoza', 'vedlajsie_diagnozy'])

#### Vykony

In [42]:
# Ako vykony pouzi vykony z 02 a ponechaj pre analyticke ucely vykony z 01

final = final.rename(columns={'vykony_02': 'vykony', 'vykony_01': 'vykony_z_01'}) 

## Pripojenie dalsich hospitalizacnych pripadov z UZS_JZS

In [43]:
# Vyber iba tie hp, ktoré sa nenachádzajú v drg dátach
# uz nebudeme potrebovat stlpec s hlavnou diagnozou

zjednotene_uzs = zjednotene_uzs.reset_index()
_doplnkove_uzs = zjednotene_uzs[~zjednotene_uzs['id_hp'].isin(final['id_hp'])]
_doplnkove_uzs = _doplnkove_uzs.drop(columns=['hlavna_diagnoza'])
_doplnkove_uzs[['diagnozy_z_01', 'vykony_z_01']] = _doplnkove_uzs[['diagnozy', 'vykony']]

len(_doplnkove_uzs)

396645

In [44]:
final = pd.concat([final, _doplnkove_uzs]).reset_index()

print(len(final))
print(final.nunique())

1194664
index                 1115616
kod_zp                      3
id_hp_pzs              113097
id_hp                 1194664
pzs_6                      90
osetrovacia_doba          172
vek_dni                   367
vek_roky                  104
hmotnost                 2177
upv                       992
datum_narodenia         35593
drg                      1144
erv                     14719
zlucene_hp               8589
vykony                 552319
obdobie_od                 18
obdobie_do                 13
vykony_z_01             32880
diagnozy_z_01           96512
id_poistenca           716566
typ_starostlivosti          4
pzs_12                   1993
datum_od               258582
datum_do               160193
diagnozy               427110
dtype: int64


## finalna uprava vsetkych hospitalizacii

### Pripojenie dat zo suboru 09_UZS_POISTENCI

In [45]:
poistenci = pd.concat([_data[i]['poistenci'] for i in _spracovat], ignore_index=True)

len(poistenci)

8014842

In [46]:
# vyhod duplicity
poistenci = poistenci.drop_duplicates()

len(poistenci)

6420893

In [47]:
# Najdi poistencov, ktori maju viac ako jeden datum narodenia a vyhod ich

# Pozn. možno bude nutné spätne ponechať tzv. fiktivnych poistencov, su to id_poistenca, u ktorych sa moze objavit viacnasobny datum narodenia, pretoze sa pouziju pre viacerych pacientov, ktorych poistenie je nezname alebo utajene. Ak takí existujú, ich zoznam sa nachádza v súbore poznamky.txt pri pri ostatných súboroch od poisťovne.   

_pocty = poistenci.groupby(['kod_zp', 'id_poistenca'])['datum_narodenia'].nunique()
_i = _pocty.iloc[np.where(_pocty > 1)[0]].index

poistenci = poistenci.set_index(['kod_zp', 'id_poistenca'])
poistenci.loc[_i].to_csv(_paths['validacia'] / 'poistenci_s_viac_datumami_narodenia.csv', **{**_argumenty_reportu, 'index':True})
poistenci = poistenci.drop(_i).reset_index()

print(len(poistenci))
print(poistenci.nunique())

6420893
kod_zp                          3
id_poistenca              6417284
datum_narodenia             41231
pohlavie                        2
kod_prechodneho_pobytu       3681
psc_prechodneho_pobytu       2839
kod_trvaleho_pobytu         23197
psc_trvaleho_pobytu         22192
dtype: int64


In [48]:
# Vytvor kod pobytu primarne z prechodneho pobytu

poistenci['kod_pobytu'] = poistenci['kod_prechodneho_pobytu'].fillna(poistenci['kod_trvaleho_pobytu'])
poistenci['psc_pobytu'] = poistenci['psc_prechodneho_pobytu'].fillna(poistenci['psc_trvaleho_pobytu'])

poistenci = poistenci.drop(columns=['kod_prechodneho_pobytu', 'kod_trvaleho_pobytu', 'psc_prechodneho_pobytu', 'psc_trvaleho_pobytu'])

In [49]:
# Vyber posledny zaznamenany pobyt
poistenci = poistenci.groupby(by=['kod_zp', 'id_poistenca', 'pohlavie', 'datum_narodenia'])[['psc_pobytu', 'kod_pobytu']].last().reset_index()

print(len(poistenci)) 
print(poistenci.nunique())

6417286
kod_zp                   3
id_poistenca       6417284
pohlavie                 2
datum_narodenia      41231
psc_pobytu           12318
kod_pobytu           12925
dtype: int64


In [50]:
# Vytvor exporty vyfiltrovaných poistencov

poistenci.to_csv(_paths['vystupy'] / f'poistenci_vyfiltrovani_{_rok}.csv', **_argumenty_reportu)

In [51]:
# Pripoj data zo suboru 09_UZS_POISTENCI

final = final.merge(poistenci, how='left', on=['kod_zp', 'id_poistenca'], suffixes=['_02', '_09'])

In [52]:
# Expotuj ID poistencov, ktore mame v datach o hospitalizaciach ale nie v datach o poistencoch

final[~final['id_poistenca'].isin(poistenci['id_poistenca'])][_reportovane_stlpce + ['id_poistenca']].to_csv(_paths['validacia'] / 'id_poistenca_nedohladane.csv', **_argumenty_reportu)

In [53]:
# vyexportuj pripady, kde sa nezhoduju datumy narodenia, a datum z 02 nie je novorodenec (datum narodenia skor ako v decembri minuleho roka)
# zjednot datumy narodenia, primarny je z 09

_rozdielne_datumy = final[['datum_narodenia_02', 'datum_narodenia_09']].notna().all(axis=1) & (final['datum_narodenia_02'] != final['datum_narodenia_09']) & (final['datum_narodenia_02'] < pd.Timestamp(year=_rok-1, month=12, day=1))

final[_rozdielne_datumy][_reportovane_stlpce + ['datum_narodenia_02', 'datum_narodenia_09']].to_csv(_paths['validacia'] / 'datum_narodenia_rozdielny.csv', **_argumenty_reportu)

final['datum_narodenia'] = final['datum_narodenia_09'].fillna(final['datum_narodenia_02'])

final = final.drop(columns=['datum_narodenia_02', 'datum_narodenia_09'])

### Kod poskytovatela pzs_8

In [54]:
# Pridaj kod nemocnice podla prevodovnika na zaklade kodov oddeleni.

_pzs_12_prevodnik = nacitaj_prevodove_subory('pzs_12')

final = final.merge(_pzs_12_prevodnik, how='left', on='pzs_12')

len(final)

1194664

In [55]:
# Exportuj kody oddeleni, ktorym nevieme priradit kod nemocnice

final[final['pzs_8'].isna()][_reportovane_stlpce + ['pzs_12', 'pzs_8']].to_csv(_paths['validacia'] / 'pzs_12_nedohladane_v_prevodniku.csv', **_argumenty_reportu)
final[final['pzs_8'].fillna('xxxxxxxx').str.endswith('xx')][_reportovane_stlpce + ['pzs_12', 'pzs_8']].to_csv(_paths['validacia'] / 'pzs_12_nedohladane_v_datach_nczi.csv', **_argumenty_reportu)

In [56]:
final['pzs_6'] = final['pzs_6_x'].fillna(final['pzs_6_y'])

final = final.drop(columns=['pzs_6_x', 'pzs_6_y'])

In [57]:
# Ak sa nepripojil kod nemocnice, lebo hospitalizacia nema zaznam v subore
# 03_HP_PREKLADY, vytvor kod nemocnice ako 'KOD_PZS + 0 + Y', kde Y sa zisti
# z polozky ID_HP_PZS ako (Z)RRYXXXXX. Y by nikdy nemalo byt 0, ale nemocnice
# si ho tak niekedy udavaju, takze sprav zmenu na 1.

_y=final['id_hp_pzs'].str.extract('^Z?\d\d(\d)').replace('0', '1')

final['pzs_8'] = final['pzs_8'].fillna(final['pzs_6'] + '0' + _y[0])

### Vyradenie hospitalizacii nekonciacich vo vybranom roku

In [58]:
_konci_v_zlom_obdobi = ((final['datum_do'].notna()) & (final['datum_do'].dt.year != _rok))

final[_konci_v_zlom_obdobi][_reportovane_stlpce + ['datum_do', 'obdobie_od', 'obdobie_do']].to_csv(_paths['validacia'] / 'konci_v_zlom_obdobi.csv', **_argumenty_reportu)

final = final.drop(final[_konci_v_zlom_obdobi].index)

len(final)

1194663

### Dlzka osetrovacej doby

In [59]:
# Napocitaj dlzku osetrovacej doby ako rozdiel v dnoch medzi datumom prepustenia a datumom prijatia. Pouzi v pripade, ze nebola dlzka osetrovacej doby reportovana.

def vypocitaj_dlzku_osetrovacej_doby(datum_prijatia, datum_prepustenia):
    if datum_prepustenia < datum_prijatia:
        return 1
    return max(1,(datum_prepustenia - datum_prijatia).days)


_vypocitana_dlzka_osetrovania = final.apply(lambda row: vypocitaj_dlzku_osetrovacej_doby(row['datum_od'], row['datum_do']), axis=1).astype('Int16')

In [60]:
final[final['osetrovacia_doba'].notna() & (final['osetrovacia_doba'] == 0)][_reportovane_stlpce + ['datum_od', 'datum_do']].to_csv(_paths['validacia'] / 'osetrovacia_doba_0.csv', **_argumenty_reportu)

final.loc[final['osetrovacia_doba'].notna() & (final['osetrovacia_doba'] == 0), 'osetrovacia_doba'] = pd.NA

In [61]:
final.loc[:,'osetrovacia_doba'] = final['osetrovacia_doba'].fillna(_vypocitana_dlzka_osetrovania)
final.loc[final['typ_starostlivosti'].isin(['JZS', 'JZS2']), 'osetrovacia_doba'] = final.loc[final['typ_starostlivosti'].isin(['JZS', 'JZS2']), 'osetrovacia_doba'].fillna(1)

### Vek

In [62]:
# Natipuj vek podla datumov hospitalizacii a narodenia a toho, co bolo napisane.
# Z datumov hospitalizacii sa najde minimalny a podla neho a datumu narodenia
# sa urci vek pri hospitalizacii. 

final['datum_prijatia'] = final[['datum_od', 'datum_do']].min(axis=1)

In [63]:
# Vypocitaj vek presne zo zadanych datumov prijatia a narodenia

def vypocitaj_vek(datum_prijatia, datum_narodenia):
    if datum_prijatia < datum_narodenia:
        return 0, 1
    vek_roky = datum_prijatia.year - datum_narodenia.year - ((datum_prijatia.month, datum_prijatia.day) < (datum_narodenia.month, datum_narodenia.day))
    if vek_roky == 0:
        return vek_roky, (datum_prijatia - datum_narodenia).days
    return vek_roky, pd.NA

final[['vypocitany_vek_roky', 'vypocitany_vek_dni']] = final.apply(lambda row: vypocitaj_vek(row['datum_prijatia'], row['datum_narodenia']), axis=1, result_type='expand')

In [64]:
# Pre pripady, ze je znamy datum narodenia ale neznamy datum prijatia, mozeme odhadnut vek ako jednoduchy rozdiel medzi aktualnym rokom a rokom narodenia

final['odhadnuty_vek_roky'] = final.apply(lambda row: _rok - row['datum_narodenia'].year, axis=1)

In [65]:
# Nakoniec ak je vyplneny vek v dnoch alebo hmotnost, vek je 0 rokov.

final.loc[(final['vek_dni'] > 0) | (final['hmotnost'] > 0), ['odhadnuty_vek_0']] = 0

In [66]:
# Prioritu ma zaznamenany vek v rokoch, doplna sa vypocitanym alebo odhadnutym vekom

final['vek_roky'] = final['vek_roky'].fillna(final['odhadnuty_vek_0'])
final['vek_roky'] = final['vek_roky'].fillna(final['vypocitany_vek_roky'])
final['vek_dni'] = final['vek_dni'].fillna(final['vypocitany_vek_dni'])
final['vek_roky'] = final['vek_roky'].fillna(final['odhadnuty_vek_roky'])


final = final.drop(
    columns=['datum_prijatia', 'vypocitany_vek_roky', 'vypocitany_vek_dni', 'odhadnuty_vek_roky', 'odhadnuty_vek_0']
    )

In [67]:
# Exportuj zaznamy, ktore maju nestandardne hodnoty
# Vysoky vek
final[final['vek_roky'] > 109][_reportovane_stlpce + ['id_poistenca', 'vek_roky']].to_csv(_paths['validacia'] / 'vek_vysoky.csv', **_argumenty_reportu)
# Novorodenci bez hmotnosti (dopln dohodnutu hodnotu 2500)
final[(final['vek_roky'] == 0) & (final['vek_dni'] <= 28) & (final['hmotnost'].isna())][_reportovane_stlpce + ['id_poistenca', 'vek_roky', 'vek_dni', 'hmotnost']].to_csv(_paths['validacia'] / 'novorodenci_bez_hmotnosti.csv', **_argumenty_reportu)

final.loc[(final['vek_roky'] == 0) & ((final['vek_dni'] <= 28) | final['vek_dni'].isna()) & (final['hmotnost'].isna()), 'hmotnost'] = 2500
# Chybajuci vek (aj napriek carovaniu so vsetkym moznym)
final[final['vek_roky'].isna()][_reportovane_stlpce + ['id_poistenca', 'vek_roky']].to_csv(_paths['validacia'] / 'vek_chybajuci.csv', **_argumenty_reportu)

In [68]:
final['vek_roky'] = pd.to_numeric(final['vek_roky'], errors='coerce').astype('Int16')
final['vek_dni'] = pd.to_numeric(final['vek_dni'], errors='coerce').astype('Int16')

### Vylucenie hospitalizacii, ktore boli zlucene kvoli DRG

In [69]:
# Zo stlpca s ciastkovymi hospitalizaciami odstran medzery (2021 ich nema, ale
# radsej pre isototu to treba spravit) a ak je ako oddelovac pouzita bodka
# (mala by byt pouzivana len ciarka, ak nie je treba to vratit poistovni)
# zmen ju na ciarku

final['zlucene_hp'] = final['zlucene_hp'].str.replace(' ', "").str.replace('.', ',')

In [70]:
# Vyber riadky, ktore maju nieco v ciastkovej hospitalizacii
_zlucene = final[final['zlucene_hp'].notna()][['pzs_8', 'zlucene_hp']].drop_duplicates()

In [71]:
# Rozdel ciastkove hospitalizacie samostatne do riadkov

_zlucene = _zlucene.set_index(['pzs_8']).apply(lambda col: col.str.split(',').explode()).reset_index()

In [72]:
# Vytvor jedinecne id pre ciastkove hospitalizacie
_zlucene['id_hp_pzs_pzs_8'] = _zlucene['zlucene_hp'] + '_' + _zlucene['pzs_8']

In [73]:
# Najdi id hospitalizacii prisluchajucich jedinecnym id

final['id_hp_pzs_pzs_8'] = final['id_hp_pzs'] + '_' + final['pzs_8']

_id_na_vylucenie = final[final['id_hp_pzs_pzs_8'].isin(_zlucene['id_hp_pzs_pzs_8'])]['id_hp']

In [74]:
# Exportuj ciastkove hospitalizacie
final[final['id_hp'].isin(_id_na_vylucenie)][_reportovane_stlpce].to_csv(_paths['validacia'] / 'zlucene_hospitalizacie.csv', **_argumenty_reportu)

In [75]:
# Vyhod ciastkove hospitalizacie

final = final.drop(final[final['id_hp'].isin(_id_na_vylucenie)].index)

final = final.drop(columns=['zlucene_hp', 'id_hp_pzs_pzs_8'])

len(final)

1178065

### Reporting chybajucich dat

In [76]:
final = final.replace('', np.nan)
_nove_reportovane_stlpce = ['kod_zp','id_hp','id_poistenca','datum_od','datum_do','typ_starostlivosti', 'obdobie_od', 'obdobie_do']

final[final['id_poistenca'].isna()][_nove_reportovane_stlpce].to_csv(_paths['validacia'] / 'id_poistenca_chybajuce.csv', **_argumenty_reportu)

final[final['typ_starostlivosti'].isna()][_nove_reportovane_stlpce + ['typ_starostlivosti']].to_csv(_paths['validacia'] / 'typ_starostlivosti_chybajuci.csv', **_argumenty_reportu)

final[final['pzs_12'].isna()][_nove_reportovane_stlpce + ['pzs_12']].to_csv(_paths['validacia'] / 'pzs_12_chybajuce.csv', **_argumenty_reportu)

final[(final['datum_od'].isna()) | (final['datum_do'].isna() & (~final['typ_starostlivosti'].isin(['JZS', 'JZS2'])))][_nove_reportovane_stlpce].to_csv(_paths['validacia'] / 'datumy_chybajuce.csv', **_argumenty_reportu)

final[final['diagnozy'].str.startswith('~', na=False)][_nove_reportovane_stlpce + ['diagnozy']].to_csv(_paths['validacia'] / 'hlavne_diagnozy_chybajuce.csv', **_argumenty_reportu)
final[final['diagnozy'].isna()][_nove_reportovane_stlpce + ['diagnozy']].to_csv(_paths['validacia'] / 'diagnozy_chybajuce.csv', **_argumenty_reportu)

## Upratanie finalnych dat

In [77]:
# Popresuvaj stlpce a vypis statistiky
final = final[['id_hp', 'vek_roky', 'vek_dni', 'hmotnost', 'upv', 'diagnozy', 'vykony', 'drg', 'erv', 'typ_starostlivosti', 'diagnozy_z_01', 'vykony_z_01', 'obdobie_od', 'obdobie_do', 'datum_od', 'datum_do', 'osetrovacia_doba', 'id_poistenca', 'datum_narodenia', 'kod_pobytu', 'kod_zp', 'pzs_12', 'IDENTIFZAR', 'pzs_8', 'pzs_6', 'id_hp_pzs']]

print(len(final))
print(final.nunique())

1178065
id_hp                 1178065
vek_roky                  106
vek_dni                   367
hmotnost                 2168
upv                       982
diagnozy               417641
vykony                 542241
drg                      1143
erv                     14438
typ_starostlivosti          4
diagnozy_z_01           95328
vykony_z_01             32737
obdobie_od                 18
obdobie_do                 13
datum_od               257089
datum_do               159072
osetrovacia_doba          378
id_poistenca           716566
datum_narodenia         35671
kod_pobytu               3733
kod_zp                      3
pzs_12                   1993
IDENTIFZAR                373
pzs_8                     417
pzs_6                     279
id_hp_pzs              112404
dtype: int64


In [78]:
# Exportuj do csv

final.to_csv(_paths['vystupy'] / f'osn_vsetka_starostlivost_{_rok}.csv', sep=';', index=False)

## Priprava dat pre algoritmus

In [79]:
# Pridaj stlpec s odbornostami (aktualne prazdny) a vyber iba chcene stlpce

_verzia_algoritmu = 'v2024.2'

if _verzia_algoritmu == 'v2024.1':
    _for_algorithm = final[['id_hp', 'vek_roky', 'vek_dni', 'hmotnost', 'upv', 'diagnozy', 'vykony', 'drg']]
    _for_algorithm.insert(7, "odbornosti", pd.NA)
            
    _doplneny_vek_dni = _for_algorithm[_for_algorithm['vek_roky'] >= 1]['vek_dni'].fillna(0)
    _for_algorithm.loc[_doplneny_vek_dni.index,['vek_dni']] = _doplneny_vek_dni
elif _verzia_algoritmu == 'v2024.2':
    _for_algorithm = final[['id_hp', 'vek_roky', 'hmotnost', 'upv', 'diagnozy', 'vykony', 'drg']]


In [80]:
# Dopln dalsie numericke hodnoty na 0

_doplnena_hmotnost = _for_algorithm[~(_for_algorithm['vek_roky'] == 0)]['hmotnost'].fillna(0)
_for_algorithm.loc[_doplnena_hmotnost.index,['hmotnost']] = _doplnena_hmotnost

_for_algorithm.loc[:, ['upv']] = _for_algorithm['upv'].fillna(0)

In [81]:
# Vyexportuj do csv

print(len(_for_algorithm))

print(_for_algorithm.nunique())

_for_algorithm.to_csv(_paths['vystupy'] / f'osn_vsetka_starostlivost_{_rok}_pre_algoritmus_{_verzia_algoritmu}.csv', sep=';', header=False, index=False)


1178065
id_hp       1178065
vek_roky        106
hmotnost       2168
upv             982
diagnozy     417641
vykony       542241
drg            1143
dtype: int64


# Rozdeľ validačné výstupy podľa poisťovní

In [82]:
for _kod in _spracovat:
    p = _paths['validacia'] / f'{_kod}'
    p.mkdir()

for file in _paths['validacia'].iterdir():
    if file.is_dir():
        continue
    df = pd.read_csv(file, sep=';', dtype='str')
    for _kod in _spracovat:
        zp_df = df[df['kod_zp'] == f'{_kod}']
        if not zp_df.empty:
            zp_df.to_csv(_paths['validacia'] / f'{_kod}' / f'{_kod}_{file.stem}.csv', index=False, sep=';')

# Validácia s tabuľkou 13

In [83]:
# Najdi relevantne tabuľky
sumar = pd.concat([_data[k]['sumar'] for k in _spracovat], ignore_index=True)
sumar['pocet'] = sumar['pocet'].astype('Int32')

len(sumar)

1057

In [84]:
# Vytvor sumar z final v rovnakom tvare, ako je vykazovany poistovanmi v tabulke 13 a porovnaj

_final_sumar = final.groupby(by=['kod_zp', 'pzs_6', 'typ_starostlivosti'])['id_hp'].count().astype('Int32').rename('pocet')

sumar = sumar.merge(_final_sumar, on=['kod_zp', 'pzs_6', 'typ_starostlivosti'], how='outer', suffixes=['_vykazany', '_spocitany'])

sumar['rozdiel_absolutny'] = sumar['pocet_vykazany'].fillna(0) - sumar['pocet_spocitany'].fillna(0)
sumar['rozdiel_relativny'] = (sumar['rozdiel_absolutny'] / sumar[['pocet_spocitany', 'pocet_vykazany']].max(axis=1)).abs()
sumar = sumar.set_index(['kod_zp', 'pzs_6', 'typ_starostlivosti'])

In [85]:
sumar.to_csv(_paths['validacia'] / 'sumar_porovnanie.csv', **{**_argumenty_reportu, 'index': True})
sumar[sumar['rozdiel_relativny'] > 0.05].to_csv(_paths['validacia'] / 'sumar_vysoke_rozdiely.csv', **{**_argumenty_reportu, 'index': True})

In [86]:
sumar.groupby(['typ_starostlivosti', 'kod_zp'])['rozdiel_absolutny'].sum().reset_index()

,typ_starostlivosti,kod_zp,rozdiel_absolutny
0,DRG,24,5564
1,DRG,25,7621
2,DRG,27,-2668
3,JZS,24,-141
4,JZS,25,0
5,JZS,27,0
6,JZS2,24,0
7,UH,24,647
8,UH,25,1
9,UH,27,0


In [87]:
# Ako by sa zmenili pocty, pokial by sa pripocitali aj vylucene HP 

sumar.groupby(['typ_starostlivosti', 'kod_zp'])['rozdiel_absolutny'].sum() - drg[drg['id_hp'].isin(_id_na_vylucenie)].groupby(['typ_starostlivosti', 'kod_zp'])['id_hp'].count().fillna(0)

typ_starostlivosti  kod_zp
DRG                 24           3.0
                    25       -1952.0
                    27       -4132.0
JZS                 24          <NA>
                    25          <NA>
                    27          <NA>
JZS2                24          <NA>
UH                  24          <NA>
                    25          <NA>
                    27          <NA>
dtype: Float64